# Classical regression models

* KNN
* Kernel SVM
* Random Forest

In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from training.creating_dataset import load_and_preprocess_data
from training.constants import FEATURES, DATASET_FILES, ORDERED_CHARACTERISTICS_FULL


bestiaries = load_and_preprocess_data(
    [f"../../pathfinder_2e_remaster_data/{f}" for f in DATASET_FILES], FEATURES
)

columns = [col for col in bestiaries.columns if col not in ["book", "level"]]
scaler = MinMaxScaler()
min_max_df = pd.DataFrame()
min_max_df[columns] = pd.DataFrame(
    scaler.fit_transform(bestiaries[columns]), index=bestiaries.index
)
min_max_df["book"] = bestiaries["book"]
min_max_df["level"] = bestiaries["level"]
bestiaries = min_max_df
bestiaries = bestiaries[ORDERED_CHARACTERISTICS_FULL + ["book", "level"]]
bestiaries.head()

,str,dex,con,int,wis,cha,ac,hp,perception,fortitude,...,area-damage_weakness,cold_weakness,cold-iron_weakness,evil_weakness,fire_weakness,good_weakness,slashing_weakness,splash-damage_weakness,book,level
0,0.588235,0.588235,0.357143,0.200000,0.470588,0.529412,0.509434,0.252087,0.156522,0.425532,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pathfinder #164: Hands of the Devil,8
1,0.588235,0.411765,0.428571,0.600000,0.411765,0.411765,0.396226,0.123539,0.130435,0.297872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pathfinder #163: Ruins of Gauntlight,5
2,0.529412,0.470588,0.214286,0.400000,0.470588,0.352941,0.320755,0.078464,0.069565,0.170213,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pathfinder #163: Ruins of Gauntlight,3
3,0.352941,0.529412,0.214286,0.333333,0.470588,0.352941,0.358491,0.048414,0.104348,0.170213,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pathfinder #164: Hands of the Devil,3
4,0.588235,0.588235,0.500000,0.466667,0.529412,0.588235,0.490566,0.198664,0.156522,0.361702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pathfinder #164: Hands of the Devil,8


In [2]:
import os
from training.splitting_dataset import split_dataframe, get_date_books_mapping

os.chdir("../../training")
books_dates_map = get_date_books_mapping()

books_to_include = [
    book for _, row in books_dates_map["books"].iteritems() for book in row
]
bestiaries = bestiaries[bestiaries["book"].isin(books_to_include)]
X_train, X_test, y_train, y_test = split_dataframe(bestiaries)
os.chdir("../notebooks/models")

In [21]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV


clf = KNeighborsRegressor()

hyper_params = {
    "leaf_size": list(range(50, 100, 10)),
    "weights": ["uniform", "distance"],
    "metric": ["minkowski", "manhattan", "euclidean"],
    "n_neighbors": list(range(1, 51)),
}

model = GridSearchCV(
    estimator=clf,
    param_grid=hyper_params,
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=2,
    return_train_score=True,
    n_jobs=-1,
)
model.fit(X_train, y_train)

Fitting 5 folds for each of 1500 candidates, totalling 7500 fits


GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'leaf_size': [50, 60, 70, 80, 90],
                         'metric': ['minkowski', 'manhattan', 'euclidean'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [22]:
model.best_params_

{'leaf_size': 50,
 'metric': 'manhattan',
 'n_neighbors': 5,
 'weights': 'distance'}

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


y_pred = model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

1.2435195397196206
2.817641490858506


In [24]:
y_pred = model.predict(X_train)

print(mean_absolute_error(y_train, y_pred))
print(mean_squared_error(y_train, y_pred))

0.0
0.0


In [5]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV


svm = SVR(kernel="rbf", max_iter=10000)
hyper_params = {"C": [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5]}

model = GridSearchCV(
    estimator=svm,
    param_grid=hyper_params,
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=2,
    return_train_score=True,
    n_jobs=-1,
)
model.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\jolka\OneDrive\Dokumenty\mgr\Predictive-ML-in-TTRPG-game-design\venv\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


GridSearchCV(cv=5, estimator=SVR(max_iter=10000), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10.0, 100.0, 1000.0,
                               10000.0, 100000.0]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [6]:
model.best_params_

{'C': 10.0}

In [25]:
svm = SVR(kernel="rbf", max_iter=10000)
hyper_params = {"C": np.linspace(1, 100, num=100)}

model = GridSearchCV(
    estimator=svm,
    param_grid=hyper_params,
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=2,
    return_train_score=True,
    n_jobs=-1,
)
model.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


GridSearchCV(cv=5, estimator=SVR(max_iter=10000), n_jobs=-1,
             param_grid={'C': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100.])},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [26]:
model.best_params_

{'C': 7.0}

In [27]:
svm = SVR(kernel="rbf", max_iter=10000)
hyper_params = {"C": np.linspace(1, 10, num=100)}

model = GridSearchCV(
    estimator=svm,
    param_grid=hyper_params,
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=2,
    return_train_score=True,
    n_jobs=-1,
)
model.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


GridSearchCV(cv=5, estimator=SVR(max_iter=10000), n_jobs=-1,
             param_grid={'C': array([ 1.        ,  1.09090909,  1.18181818,  1.27272727,  1.36363636,
        1.45454545,  1.54545455,  1.63636364,  1.72727273,  1.81818182,
        1.90909091,  2.        ,  2.09090909,  2.18181818,  2.27272727,
        2.36363636,  2.45454545,  2.54545455,  2.63636364,  2.72727273,
        2.81818182,  2.90909091,  3.        ,  3.09090909,  3.18181818,
        3.27272727,  3.36363...
        7.36363636,  7.45454545,  7.54545455,  7.63636364,  7.72727273,
        7.81818182,  7.90909091,  8.        ,  8.09090909,  8.18181818,
        8.27272727,  8.36363636,  8.45454545,  8.54545455,  8.63636364,
        8.72727273,  8.81818182,  8.90909091,  9.        ,  9.09090909,
        9.18181818,  9.27272727,  9.36363636,  9.45454545,  9.54545455,
        9.63636364,  9.72727273,  9.81818182,  9.90909091, 10.        ])},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [28]:
model.best_params_

{'C': 6.363636363636364}

In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


y_pred = model.predict(X_test)

mean_absolute_error(y_test, y_pred)

0.5594337917196204

In [30]:
mean_squared_error(y_test, y_pred)

1.0202996727869298

In [31]:
y_pred = model.predict(X_train)

print(mean_absolute_error(y_train, y_pred))
print(mean_squared_error(y_train, y_pred))

0.19529077488239519
0.09230465868082151


In [37]:
from sklearn.ensemble import RandomForestRegressor
from training.constants import RANDOM_STATE


rf = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)
hyper_params = {
    # "n_estimators": [
    #     int(x) for x in np.linspace(start=100, stop=800, num=8)
    # ],
    "max_features": [0.3],
    "max_depth": [100, 200, 500],
    "criterion": ["squared_error", "absolute_error"],
}
model = GridSearchCV(
    estimator=rf,
    param_grid=hyper_params,
    scoring="neg_mean_absolute_error",
    cv=5,
    return_train_score=True,
    n_jobs=-1,
)
model.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [100, 200, 500], 'max_features': [0.3]},
             return_train_score=True, scoring='neg_mean_absolute_error')

In [38]:
model.best_params_

{'criterion': 'squared_error', 'max_depth': 100, 'max_features': 0.3}

In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


y_pred = model.predict(X_test)

mean_absolute_error(y_test, y_pred)

0.4423897058823529

In [41]:
mean_squared_error(y_test, y_pred)

0.5033687499999999

In [42]:
y_pred = model.predict(X_train)

print(mean_absolute_error(y_train, y_pred))
print(mean_squared_error(y_train, y_pred))

0.08296759941089833
0.01791509572901326
